#CEL LABORATORIUM: 
Na tym laboratorium mieliśmy dowiedzieć się jak zająć się typami danych często występującymi w rzeczywistych zestawach danych (brakujące wartości, zmienne kategorialne),projektować potoki w celu poprawy jakości kodu uczenia maszynowego,stosować zaawansowane techniki walidacji modeli (walidacja krzyżowa),budować najnowocześniejsze modele, które są powszechnie używane do wygrywania zawodów Kaggle (XGBoost), oraz uniknąć typowych i ważnych błędów w nauce o danych (wycieku).


Istnieje wiele sposobów, jakie dane mogą kończyć się brakującymi wartościami. Musisz więc wybrać jedną z poniższych opcji.

Wyróżniamy :
#1) Prosta opcja: upuszczenie kolumny z brakami wartości
#2) Lepsza opcja: przypisanie
#3) Przedłużenie przypisania


# 2. Brakujące dane

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Ładowanie danych
data = pd.read_csv('/content/melb_data.csv')

# Wybór celu
y = data.Price

# Aby uprościć sprawę, użyjemy tylko predyktorów numerycznych
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])
# Podziel dane na podzbiory treningowe i walidacyjne
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

Zdefiniuj funkcję pomiaru jakości każdego podejścia

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Funkcja do porównywania różnych podejść
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

1)Upuszczanie kolumn z brakami wartości

In [ ]:
# Pobierz nazwy kolumn z brakami danych
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
# Upuść kolumny w danych treningowych i walidacyjnych
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE z Podejścia 1 (Usuń kolumny z brakującymi wartościami):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE z Podejścia 1 (Usuń kolumny z brakującymi wartościami):
183550.22137772635


2) Przypisanie
Następnie używamy SimpleImputer, aby zastąpić brakujące wartości średnią wartością w każdej kolumnie.

In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE z Podejścia 2 (Imputacja):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE z Podejścia 2 (Imputacja):
178166.46269899711


3) Przedłużenie przypisania
Następnie imputujemy brakujące wartości, jednocześnie śledząc, które wartości zostały imputowane.

In [ ]:
# Utwórz kopię, aby uniknąć zmiany oryginalnych danych (podczas imputowania)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Utwórz nowe kolumny wskazujące, co zostanie przypisane
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputacja
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputacja usuniętych nazw kolumn; odłóż je z powrotem
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE z podejścia 3 (rozszerzenie do przypisania):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE z podejścia 3 (rozszerzenie do przypisania):
178927.503183954


Imputacja działała lepiej niż porzucanie kolumn ponieważ  
Dane szkoleniowe mają 10864 wierszy i 12 kolumn, z których trzy kolumny zawierają brakujące dane. W każdej kolumnie brakuje mniej niż połowy wpisów. W związku z tym upuszczenie kolumn powoduje usunięcie wielu przydatnych informacji, dlatego sensowne jest, aby imputacja działała lepiej.

In [ ]:
# Kształt danych szkoleniowych (num_rows, num_columns)
print(X_train.shape)

# Liczba brakujących wartości w każdej kolumnie danych szkoleniowych
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


# 3. Zmienna kategorialna

Zmienna kategorialna przyjmuje tylko ograniczoną liczbę wartości.
Rozważę ankietę, która pyta, jak często pomagasz kolegom w nauce i oferuje cztery opcje: „Nigdy”, „Rzadko”, „Większość dni” lub „Codziennie”.
Błąd pojawi się w momencie gdy podłączysz zmienne do modeli uczenia maszynowego bez ich wcześniejszego przetworzenia.

aby uniknąć błędów mamy 3 podejścia:
1) Usuń zmienne kategorialne-najłatwiejszym sposobem radzenia sobie ze zmiennymi kategorialnymi jest po prostu usunięcie ich ze zbioru danych. Takie podejście będzie działać dobrze tylko wtedy, gdy kolumny nie zawierały przydatnych informacji.
2) Kodowanie etykiet-przypisuje każdą unikalną wartość do innej liczby całkowitej.
3) Kodowanie na gorąco - tworzy nowe kolumny wskazujące na obecność (lub brak) każdej możliwej wartości w oryginalnych danych.



Przykład:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Odczyt danych
data = pd.read_csv('/content/melb_data.csv')

# Oddziel cel od predyktorów
y = data.Price
X = data.drop(['Price'], axis=1)

# Podziel dane na podzbiory treningowe i walidacyjne
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Usuń kolumny z brakującymi wartościami (najprostsze podejście)


cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# „Liczność” oznacza liczbę unikatowych wartości w kolumnie
# Wybierz kolumny kategorialne o stosunkowo niskiej liczności (wygodne, ale dowolne)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Wybierz kolumny numeryczne
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Zachowaj tylko wybrane kolumny
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Rzucamy okiem na dane treningowe za pomocą metody head () poniżej.

In [ ]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


Następnie otrzymujemy listę wszystkich zmiennych kategorialnych w danych uczących.

Robimy to, sprawdzając typ danych (lub typ) każdej kolumny. Obiekt dtype wskazuje, że kolumna zawiera tekst (są inne rzeczy, które teoretycznie mogłyby to być, ale to nie jest ważne dla naszych celów). W przypadku tego zbioru danych kolumny z tekstem wskazują zmienne kategorialne.

In [ ]:
# Pobierz listę zmiennych kategorialnych
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Zmienne kategorialne:")
print(object_cols)



Zmienne kategorialne:
['Type', 'Method', 'Regionname']


Zdefiniowanie funkcji pomiaru jakości każdego podejścia

Definiujemy funkcję score_dataset (), aby porównać trzy różne podejścia do pracy ze zmiennymi kategorialnymi. Ta funkcja raportuje średni błąd bezwzględny (MAE) z losowego modelu lasu. Ogólnie rzecz biorąc, chcemy, aby MAE było jak najniższe!

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Funkcja do porównywania różnych podejść
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

Wynik z podejścia 1 (porzuć zmienne kategorialne)

Porzucamy kolumny obiektów metodą select_dtypes ().


In [ ]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE z Podejścia 1 (Usuń zmienne kategorialne):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE z Podejścia 1 (Usuń zmienne kategorialne):
175703.48185157913


Wynik z podejścia 2 (kodowanie etykiet)

Scikit-learn ma klasę LabelEncoder, której można użyć do uzyskania kodowania etykiet. W pętli przeglądamy zmienne kategorialne i stosujemy koder etykiet oddzielnie do każdej kolumny.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Kopia, aby uniknąć zmiany oryginalnych danych
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Zastosowanie kodera etykiet do każdej kolumny z danymi kategorycznymi
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

print("MAE z podejścia 2 (kodowanie etykiet):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE z podejścia 2 (kodowanie etykiet):
165936.40548390493


Wynik z podejścia 3 (One-Hot Encoding)

Używamy klasy OneHotEncoder ze scikit-learn, aby uzyskać kodowanie na gorąco. Istnieje wiele parametrów, za pomocą których można dostosować jego zachowanie.
Ustawiliśmy handle_unknown = 'ignore', aby uniknąć błędów, gdy dane walidacyjne zawierają klasy, które nie są reprezentowane w danych szkoleniowych, oraz
ustawienie sparse = False zapewnia, że ​​zakodowane kolumny są zwracane jako tablica numpy (zamiast rzadkiej macierzy).

Aby użyć kodera, dostarczamy tylko kolumny kategorialne, które chcemy zakodować na gorąco. Na przykład, aby zakodować dane szkoleniowe, dostarczamy X_train[object_cols].(obiekt_cols w komórce kodu poniżej to lista nazw kolumn z danymi kategorialnymi, dlatego X_train [object_cols] zawiera wszystkie dane kategoryczne w zestawie uczącym).

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Zastosowanie jednego gorącego kodera do każdej kolumny z danymi kategorycznymi
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# Usunięty indeks jednego gorącego kodowania; odłóż to z powrotem
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Usuwanie kolumny kategorycznej (zastąpione jednym gorącym kodowaniem)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Dodawanie gorących zakodowanych kolumn do funkcji numerycznych
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE z Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE z Approach 3 (One-Hot Encoding):
166089.4893009678


Które podejście jest najlepsze?

W tym przypadku upuszczenie kolumn kategorycznych (Podejście 1) wypadło najgorzej, ponieważ miało najwyższy wynik MAE. Jeśli chodzi o pozostałe dwa podejścia, ponieważ zwrócone wyniki MAE mają tak bliską wartość, wydaje się, że nie ma żadnych znaczących korzyści dla jednego nad drugim.

Ogólnie rzecz biorąc, kodowanie typu one-hot (podejście 3) zazwyczaj działa najlepiej, a usunięcie kolumn kategorialnych (podejście 1) zazwyczaj działa najgorzej, ale różni się w zależności od przypadku.

# 4.Potoki

Wprowadzenie

Potoki to prosty sposób na zorganizowanie wstępnego przetwarzania danych i modelowania kodu. W szczególności potok obejmuje etapy wstępnego przetwarzania i modelowania, dzięki czemu można używać całego pakietu tak, jakby był to pojedynczy krok.

Wielu analityków danych hakuje razem modele bez rurociągów, ale rurociągi mają kilka ważnych zalet. Należą do nich:

Czystszy kod: rozliczanie danych na każdym etapie przetwarzania wstępnego może być skomplikowane. Dzięki potokowi nie musisz ręcznie śledzić danych treningowych i walidacyjnych na każdym etapie.
Mniej błędów: istnieje mniej okazji do błędnego zastosowania kroku lub zapomnienia o kroku przetwarzania wstępnego.
Łatwiejsze w produkcji: Przeniesienie modelu z prototypu na coś, co można wdrożyć na dużą skalę, może być zaskakująco trudne. Nie będziemy tu wchodzić w wiele powiązanych problemów, ale rurociągi mogą pomóc.
Więcej opcji sprawdzania poprawności modelu: W następnym samouczku zobaczysz przykład, który obejmuje weryfikację krzyżową.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Odczyt danych
data = pd.read_csv('/content/melb_data.csv')

# Oddziel cel od predyktorów
y = data.Price
X = data.drop(['Price'], axis=1)

# Podziel dane na podzbiory treningowe i walidacyjne
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# „Liczność” oznacza liczbę unikatowych wartości w kolumnie
# Wybierz kolumny kategorialne o stosunkowo niskiej liczności (wygodne, ale dowolne)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]



# Wybierz kolumny numeryczne
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Zachowaj tylko wybrane kolumny
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


Krok 1: Zdefiniuj kroki przetwarzania wstępneg

Podobnie do tego, jak potok łączy etapy przetwarzania wstępnego i modelowania, używamy klasy ColumnTransformer do łączenia różnych etapów przetwarzania wstępnego. Poniższy kod:umieszcza brakujące wartości w danych liczbowych,przypisuje brakujące wartości i stosuje kodowanie typu one-hot do danych kategorialnych.



In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Wstępne przetwarzanie danych liczbowych


numerical_transformer = SimpleImputer(strategy='constant')

# Przetwarzanie wstępne dla danych kategorycznych
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Wstępne przetwarzanie pakietów dla danych liczbowych i kategorialnych
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

Krok 2: Definiowanie modelu.

Następnie definiujemy losowy model lasu za pomocą znanej klasy RandomForestRegressor.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)



Krok 3: Utworzenie i ocena potok u

Na koniec używamy klasy Pipeline do definiowania potoku obejmującego etapy przetwarzania wstępnego i modelowania. Należy zwrócić uwagę na kilka ważnych rzeczy:
Za pomocą potoku wstępnie przetwarzamy dane szkoleniowe i dopasowujemy model w jednym wierszu kodu. (W przeciwieństwie do tego, bez potoku, musimy wykonywać imputację, kodowanie typu one-hot i trenowanie modelu w oddzielnych krokach. Staje się to szczególnie kłopotliwe, jeśli mamy do czynienia zarówno ze zmiennymi numerycznymi, jak i kategorialnymi!)
Wraz z potokiem dostarczamy nieprzetworzone funkcje w X_valid do polecenia Predict (), a potok automatycznie przetwarza funkcje przed wygenerowaniem prognoz. (Jednak bez potoku musimy pamiętać o wstępnym przetworzeniu danych walidacyjnych przed wykonaniem prognoz).

In [ ]:
from sklearn.metrics import mean_absolute_error

# Przetwarzanie wstępne i modelowanie kodu w potoku
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Wstępne przetwarzanie danych treningowych, dopasowanie modelu
my_pipeline.fit(X_train, y_train)

# Wstępne przetwarzanie danych walidacyjnych, uzyskiwanie prognoz
preds = my_pipeline.predict(X_valid)

# Ocena modelu
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855


Potoki są cenne do czyszczenia kodu uczenia maszynowego i unikania błędów oraz są szczególnie przydatne w przepływach pracy z zaawansowanym przetwarzaniem wstępnym danych.

# 5. Walidacja krzyżowa
Co to jest walidacja krzyżowa?

W ramach walidacji krzyżowej uruchamiamy proces modelowania na różnych podzbiorach danych, aby uzyskać wiele miar jakości modelu.
Będziemy pracować z tymi samymi danymi, co w poprzednim samouczku. Ładujemy dane wejściowe w X, a dane wyjściowe w Y.

In [ ]:
import pandas as pd

# Odczyt danych
data = pd.read_csv('/content/melb_data.csv')

# Wybierz podzbiór predyktorów


cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Wybór celu
y = data.Price



Następnie definiujemy potok, który używa imputera do uzupełniania brakujących wartości i losowego modelu lasu do tworzenia prognoz.

Chociaż można przeprowadzić weryfikację krzyżową bez potoków, jest to dość trudne! Użycie potoku sprawi, że kod będzie niezwykle prosty.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

Otrzymujemy wyniki walidacji krzyżowej za pomocą funkcji cross_val_score () z scikit-learn. Liczbę fałd ustalamy parametrem cv.

In [ ]:
from sklearn.model_selection import cross_val_score

# Pomnóż przez -1, ponieważ sklearn oblicza * ujemne * MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)



MAE scores:
 [301628.7893587  303164.4782723  287298.331666   236061.84754543
 260383.45111427]


Parametr punktacji wybiera miarę jakości modelu do raportu: w tym przypadku wybraliśmy ujemny średni błąd bezwzględny (MAE). Dokumentacja scikit-learn pokazuje listę opcji.

Zaskakujące jest to, że podajemy ujemne MAE. Scikit-learn ma konwencję, w której wszystkie metryki są zdefiniowane, więc wyższa liczba jest lepsza. Używanie tutaj negatywów pozwala im być zgodnymi z tą konwencją, chociaż negatywne MAE jest prawie niespotykane gdzie indziej.

Zwykle potrzebujemy jednej miary jakości modelu, aby porównać alternatywne modele. Więc bierzemy średnią z eksperymentów.

In [ ]:
print("Średni wynik MAE (we wszystkich eksperymentach):")
print(scores.mean())

Średni wynik MAE (we wszystkich eksperymentach):
277707.3795913405


Korzystanie z walidacji krzyżowej zapewnia znacznie lepszą miarę jakości modelu, z dodatkową zaletą czyszczenia naszego kodu: Trzeba pamiętać, że nie musimy już śledzić oddzielnych zestawów uczących i walidacyjnych. Tak więc, szczególnie w przypadku małych zbiorów danych, jest to dobra poprawa!

# 6. XGBoost
Przez większość tego kursu dokonywałem prognoz metodą losowego lasu, która osiąga lepszą wydajność niż pojedyncze drzewo decyzyjne, po prostu poprzez uśrednienie prognoz wielu drzew decyzyjnych.

Metodę losowego lasu nazywamy „metodą zespołową”. Z definicji metody zespołowe łączą przewidywania kilku modeli (np. Kilku drzew w przypadku lasów losowych).

Następnie przedstawiona zostanie inna metoda zespolona zwana wzmocnieniem gradientowym.
Wzmocnienie gradientowe to metoda, która przechodzi przez cykle, aby iteracyjnie dodawać modele do zespołu.

Rozpoczyna się inicjalizacją zespołu za pomocą pojedynczego modelu, którego przewidywania mogą być dość naiwne. (Nawet jeśli jego przewidywania są szalenie niedokładne, kolejne dodatki do zespołu rozwiązują te błędy).



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Odczyt danych
data = pd.read_csv('/content/melb_data.csv')

# Wybór podzbioru predyktorów
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Wybór celu
y = data.Price

# Oddzielenie danych do zbiorów uczących i walidacyjnych
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

W tym przykładzie  pracowałem z biblioteką XGBoost. XGBoost oznacza ekstremalne wzmocnienie gradientu, które jest implementacją wzmocnienia gradientu z kilkoma dodatkowymi funkcjami skoncentrowanymi na wydajności i szybkości. (Scikit-learn ma inną wersję wzmocnienia gradientu, ale XGBoost ma pewne zalety techniczne).

W następnej komórce kodu importujemy scikit-learn API dla XGBoost (xgboost.XGBRegressor). To pozwala nam budować i dopasowywać model tak, jak robilibyśmy to w scikit-learn. Jak zobaczysz na wyjściu, klasa XGBRegressor ma wiele parametrów do strojenia - dowiesz się o nich wkrótce!

In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

[18:13:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 267505.45321704715


Strojenie parametrów

XGBoost ma kilka parametrów, które mogą radykalnie wpłynąć na dokładność i szybkość treningu. Pierwsze parametry, które powiniśmy zrozumieć, to:
n_estimators

n_estimators określa, ile razy należy przejść przez cykl modelowania opisany powyżej. Jest równa liczbie modeli, które włączamy do zespołu.

Zbyt niska wartość powoduje niedopasowanie, co prowadzi do niedokładnych prognoz zarówno dotyczących danych uczących, jak i danych testowych.
Zbyt duża wartość powoduje nadmierne dopasowanie, co powoduje trafne przewidywania dotyczące danych treningowych, ale niedokładne przewidywania dotyczące danych testowych (na czym nam zależy).

Typowe wartości mieszczą się w zakresie 100-1000, chociaż zależy to w dużej mierze od parametru learning_rate omówionego poniżej.

Oto kod do ustawiania liczby modeli w zestawie:

In [ ]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)



early_stopping_rounds

early_stopping_rounds oferuje sposób na automatyczne znalezienie idealnej wartości dla n_estimators. Wczesne zatrzymanie powoduje, że model przestaje iterować, gdy wynik walidacji przestaje się poprawiać, nawet jeśli nie jesteśmy na twardym zatrzymaniu dla n_estimators. Rozsądnie jest ustawić wysoką wartość dla n_estimators, a następnie użyć early_stopping_rounds, aby znaleźć optymalny czas na zatrzymanie iteracji.

Ponieważ przypadkowa szansa czasami powoduje pojedynczą rundę, w której wyniki walidacji nie ulegają poprawie, przed zatrzymaniem należy określić liczbę, na ile rund pogorszenia jakości prostej można pozwolić. Ustawienie early_stopping_rounds = 5 jest rozsądnym wyborem. W takim przypadku zatrzymujemy się po 5 prostych rundach z pogarszającymi się wynikami walidacji.

Korzystając z early_stopping_rounds, musisz również odłożyć na bok niektóre dane do obliczenia wyników walidacji - odbywa się to poprzez ustawienie parametru eval_set.

Możemy zmodyfikować powyższy przykład, aby uwzględnić wczesne zatrzymanie:

In [ ]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

Jeśli później chcemy dopasować model do wszystkich swoich danych, ustaw n_estimators na dowolną wartość, którą uznasz za optymalną, gdy jest uruchamiany z wczesnym zatrzymaniem.
learning_rate

Zamiast uzyskiwać prognozy po prostu sumując prognozy z każdego modelu składowego, możemy pomnożyć prognozy z każdego modelu przez niewielką liczbę (znaną jako współczynnik uczenia się) przed ich dodaniem.

Oznacza to, że każde drzewo, które dodajemy do zespołu, pomaga nam mniej. Więc możemy ustawić wyższą wartość dla n_estimators bez przeuczenia. Jeśli zastosujemy wczesne zatrzymywanie, odpowiednia liczba drzew zostanie określona automatycznie.

Ogólnie rzecz biorąc, mały współczynnik uczenia się i duża liczba estymatorów dadzą dokładniejsze modele XGBoost, chociaż trenowanie modelu zajmie więcej czasu, ponieważ wykonuje on więcej iteracji w całym cyklu. Domyślnie XGBoost ustawia learning_rate = 0,1.

Modyfikacja powyższego przykładu w celu zmiany współczynnika uczenia daje następujący kod:

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)



n_jobs

W przypadku większych zbiorów danych, w których bierze się pod uwagę czas wykonywania, można użyć równoległości, aby szybciej budować modele. Często ustawia się parametr n_jobs równy liczbie rdzeni na komputerze. W przypadku mniejszych zbiorów danych to nie pomoże.

Wynikowy model nie będzie lepszy, więc mikro-optymalizacja pod kątem czasu dopasowania jest zazwyczaj niczym innym jak rozproszeniem. Jest to jednak przydatne w przypadku dużych zbiorów danych, w których w innym przypadku spędziłbym dużo czasu na oczekiwaniu na wykonanie polecenia dopasowania.

Oto zmodyfikowany przykład:

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

Wniosek

XGBoost to wiodąca biblioteka oprogramowania do pracy ze standardowymi danymi tabelarycznymi (typ danych przechowywanych w Pandas DataFrames, w przeciwieństwie do bardziej egzotycznych typów danych, takich jak obrazy i filmy). Dzięki starannemu dostrajaniu parametrów można trenować bardzo dokładne modele.

# 7. Wyciek danych
Wyciek danych (lub wyciek) ma miejsce, gdy dane szkoleniowe zawierają informacje o celu, ale podobne dane nie będą dostępne, gdy model jest używany do prognozowania. Prowadzi to do wysokiej wydajności zbioru uczącego (i być może nawet danych walidacyjnych), ale model będzie działał słabo w produkcji.

Innymi słowy, wyciek powoduje, że model wygląda dokładnie, dopóki nie zaczniesz podejmować decyzji z modelem, a wtedy model staje się bardzo niedokładny.

Istnieją dwa główne rodzaje wycieków: wyciek docelowy i zanieczyszczenie podczas testu treningu:
-Docelowy wyciek-występuje, gdy predyktory zawierają dane, które nie będą dostępne w momencie tworzenia prognoz.-Zanieczyszczenie testem treningowym-inny typ wycieku występuje, gdy nie jesteśmy ostrożni przy odróżnianiu danych treningowych od danych walidacyjnych.

Przykład

W tym przykładzie pokazany jest sposób wykrywania i usuwania docelowego wycieku.

Wykorzystamy zbiór danych o aplikacjach do kart kredytowych i pominiemy podstawowy kod konfiguracji danych. W rezultacie informacje o każdym wniosku o kartę kredytową są przechowywane w DataFrame X. Wykorzystamy je do przewidzenia, które wnioski zostały zaakceptowane w serii Y.



In [ ]:


import pandas as pd

# Read the data
data = pd.read_csv('/content/AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])

# Select target
y = data.card

# Select predictors
X = data.drop(['card'], axis=1)

print("Liczba wierszy w zbiorze danych:", X.shape[0])
X.head()



Liczba wierszy w zbiorze danych: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


Ponieważ jest to mały zbiór danych, użyjemy walidacji krzyżowej, aby zapewnić dokładne pomiary jakości modelu.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Since there is no preprocessing, we don't need a pipeline (used anyway as best practice!)
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation precyzja: %f" % cv_scores.mean())



Cross-validation precyzja: 0.981049


Kilka zmiennych wygląda podejrzanie. Na przykład, czy wydatek oznacza wydatek na tę kartę lub na karty użyte przed złożeniem wniosku?

W tym momencie podstawowe porównania danych mogą być bardzo pomocne:

In [ ]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Odsetek osób, które nie otrzymały karty i nie poniosły żadnych wydatków: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Odsetek tych, którzy otrzymali kartę i nie ponieśli żadnych wydatków: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Odsetek osób, które nie otrzymały karty i nie poniosły żadnych wydatków: 1.00
Odsetek tych, którzy otrzymali kartę i nie ponieśli żadnych wydatków: 0.02


Jak pokazano powyżej, każdy, kto nie otrzymał karty, nie ponosił żadnych wydatków, podczas gdy tylko 2% osób, które otrzymały kartę, nie miało żadnych wydatków. Nic dziwnego, że nasz model wydawał się mieć dużą dokładność. Ale wydaje się, że jest to również przypadek wycieku docelowego, gdzie wydatki prawdopodobnie oznaczają wydatki na kartę, o którą się ubiegali.

Ponieważ udział częściowo zależy od wydatków, należy go również wykluczyć. Zmienne aktywne i karty główne są trochę mniej jasne, ale z opisu brzmią niepokojąco. W większości sytuacji lepiej być bezpiecznym niż żałować, jeśli nie możesz wyśledzić osób, które utworzyły dane, aby dowiedzieć się więcej.

Uruchomilibyśmy model bez docelowego wycieku w następujący sposób:

In [ ]:
# Usuwanie nieszczelnych predyktorów ze zbioru danych
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Ocena modelu z usuniętymi nieszczelnymi predyktorami
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-val precyzja: %f" % cv_scores.mean())



Cross-val precyzja: 0.830934


Ta dokładność jest nieco niższa, co może być rozczarowujące. Możemy jednak oczekiwać, że w około 80% przypadków będzie to poprawne, gdy będzie używane w nowych aplikacjach, podczas gdy nieszczelny model prawdopodobnie wypadłby znacznie gorzej (pomimo wyższego pozornego wyniku w walidacji krzyżowej).

Wyciek danych może być wielomilionowym błędem w wielu aplikacjach do nauki o danych. Staranne rozdzielenie danych treningowych i walidacyjnych może zapobiec zanieczyszczeniu testów pociągu, a rurociągi mogą pomóc we wdrożeniu tego oddzielenia. Podobnie, połączenie ostrożności, zdrowego rozsądku i eksploracji danych może pomóc zidentyfikować wyciek docelowy.